In [1]:
!pip install -U textblob
!python -m textblob.download_corpora


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [2]:
import pandas as pd
import numpy as np
import regex as re
import matplotlib.pyplot as plt
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from sklearn.model_selection import train_test_split
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
from wordcloud import WordCloud,STOPWORDS
nltk.download("stopwords")
stopwords_set = set(stopwords.words("english"))
pd.options.mode.chained_assignment = None


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
df = pd.read_csv("/content/stress_detection.csv")
df.head(5)

,subreddit,post_id,sentence_range,text,id,label,confidence,social_timestamp,social_karma,syntax_ari,...,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",33181,1,0.8,1521614353,5,1.806818,...,1.000,1.1250,1.0,1.77000,1.52211,1.89556,0.86,1,3.253573,-0.002742
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",2606,0,1.0,1527009817,4,9.429737,...,1.125,1.0000,1.0,1.69586,1.62045,1.88919,0.65,2,8.828316,0.292857
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,38816,1,0.8,1535935605,2,7.769821,...,1.000,1.1429,1.0,1.83088,1.58108,1.85828,0.67,0,7.841667,0.011894
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",239,1,0.6,1516429555,0,2.667798,...,1.000,1.1250,1.0,1.75356,1.52114,1.98848,0.50,5,4.104027,0.141671
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1421,1,0.8,1539809005,24,7.554238,...,1.000,1.1250,1.0,1.77644,1.64872,1.81456,1.00,1,7.910952,-0.204167


In [5]:
stopwords_list = set(stopwords.words("english"))

no_stopwords = ["not", "don't", 'aren', 'don', 'ain', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't",
                'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't",
                "won't", 'wouldn', "wouldn't"]

for no_stopword in no_stopwords:
    stopwords_list.remove(no_stopword)

print(f'Final list of stopwords:\n{stopwords_list}')


Final list of stopwords:
{'here', 'so', 'yourselves', 'those', 'was', 'which', 'who', 'such', 'where', 'too', 'ourselves', 'what', 'but', 'this', 'should', 'yours', 'out', "she's", 'theirs', 'up', 'on', 'my', 's', 've', 'yourself', 'than', 'himself', 'each', 'myself', 'itself', 'during', 'by', 'once', 'hers', 'further', 'do', 'whom', 'are', 'o', 'being', 'an', 'until', 'is', 'a', 'into', 'any', 'some', 'the', 'can', 'then', 'y', 'few', 'only', 'your', 't', 'for', 'they', 'no', 'as', 'with', 'at', 'between', 'his', 'you', 'over', 'why', 'll', 'them', 'doing', 'while', 'nor', "you'd", 'just', 'how', 'there', 'because', 'she', 'me', 'him', 'been', 'has', 'ours', "you'll", 'did', 'and', 'from', 'below', 'when', 'most', 'own', 'am', 'their', 'were', 'same', "should've", 'off', 'again', 'before', 'now', "you've", 'through', 're', 'have', 'it', 'd', 'in', 'of', "that'll", 'more', 'other', 'against', 'be', 'that', 'or', 'm', 'after', 'if', "it's", "you're", 'our', 'won', 'her', 'very', 'will',

In [6]:
re_negation = re.compile("n't ")

def negation_abbreviated_to_standard(sent):
    sent = re_negation.sub(" not ", sent)
    return sent

word_example = "I aren't "
print(f'The sentence "{word_example}" is transformed to "{negation_abbreviated_to_standard(word_example)}"')


The sentence "I aren't " is transformed to "I are not "


In [7]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemma_stem_text(words_list):
    text = [lemmatizer.lemmatize(token.lower()) for token in words_list]
    text = [lemmatizer.lemmatize(token.lower(), "v") for token in text]
    return text

word_example = "Killing"
print(f'The word "{word_example}" is transformed to "{lemma_stem_text([word_example])[0]}"')


The word "Killing" is transformed to "kill"


In [8]:
def clean(review_text):
    review_text = negation_abbreviated_to_standard(review_text)
    letters_numbers_only = re.sub("[^a-zA-Z_0-9]", " ", review_text)
    words = np.char.lower(letters_numbers_only.split())
    meaningful_words = [w for w in words if not w in stopwords_list]
    lemma_words = lemma_stem_text(meaningful_words)
    return " ".join(lemma_words)


In [9]:
for i , j in enumerate(df['text'].tolist()):
  df['text'][i] = clean(j)

In [10]:
train, test = train_test_split(df, test_size=0.1)
train = train[train.sentiment != 0]


In [11]:
tweets = []
for index, row in train.iterrows():
    words_filtered = [e.lower() for e in row.text.split() if len(e) >= 3]
    words_cleaned = [word for word in words_filtered
                     if 'http' not in word
                     and not word.startswith('@')
                     and not word.startswith('#')
                     and word != 'RT']
    words_without_stopwords = [word for word in words_cleaned if not word in stopwords_set]
    tweets.append((words_without_stopwords, row.sentiment))


In [12]:
def get_words_in_tweets(tweets):
    all = []
    for (words, sentiment) in tweets:
        all.extend(words)
    return all

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    features = wordlist.keys()
    return features

w_features = get_word_features(get_words_in_tweets(tweets))

def extract_features(document):
    document_words = set(document)
    features = {}
    for word in w_features:
        features['contains(%s)' % word] = (word in document_words)
    return features


In [13]:
def wordcloud_draw(data, color = 'black'):            #this function generates the worldcloud from the given data using Scikit Learns Function
    words = ' '.join(data)
    cleaned_word = " ".join([word for word in words.split()
                            if 'http' not in word
                                and not word.startswith('@')
                                and not word.startswith('#')
                                and word != 'RT'
                            ])
    wordcloud = WordCloud(stopwords=STOPWORDS,
                      background_color=color,
                      width=2500,
                      height=2000
                     ).generate(cleaned_word)
    plt.figure(1)
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()

In [14]:
pos = [j for i, j in enumerate(df['text']) if df['sentiment'][i] == 1]
neg = [j for i, j in enumerate(df['text']) if df['sentiment'][i] == -1]


In [15]:
training_set = nltk.classify.apply_features(extract_features,tweets)
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [ ]:
y_test = test['sentiment']
y_pred = []
for obj in test['text'].tolist():
    res = classifier.classify(extract_features(obj.split()))
    if res == 0:
        y_pred.append(1)
    else:
        y_pred.append(res)


In [ ]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

In [ ]:
import seaborn as sns
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

In [ ]:
print(df['text'][1])
print(df['sentiment'][1])
Tweet = TextBlob(df['text'][1])
Tweet.sentiment[0]


In [ ]:
y_true = df['sentiment'].tolist()
y_pred_tb = []
for i , j in enumerate(df['text'].tolist()):
  if int(TextBlob(j).sentiment[0]) >= 0:
    y_pred_tb.append(1)
  else :
    y_pred_tb.append(-1)

In [ ]:
cnf_matrix = metrics.confusion_matrix(y_true, y_pred_tb)
cnf_matrix

In [ ]:
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_true, y_pred_tb))
print("Precision:",metrics.precision_score(y_true, y_pred_tb))
print("Recall:",metrics.recall_score(y_true, y_pred_tb))

In [ ]:
x_train = train['text']
y_train = train['sentiment']
x_test = test['text']
y_test = test['sentiment']



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=20000, ngram_range=(1, 2))
train_data_features = vectorizer.fit_transform(x_train)

print(train_data_features.shape)


In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
model.fit(train_data_features, y_train)


In [ ]:
test_data_features = vectorizer.transform(x_test)
result = model.predict(test_data_features)
y_pred_LR = result
y_test = test['sentiment']


In [ ]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred_LR)
cnf_matrix

In [ ]:
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True,  color = 'Red'  ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_LR))
print("Precision:",metrics.precision_score(y_test, y_pred_LR))
print("Recall:",metrics.recall_score(y_test, y_pred_LR))